In [54]:
from backtest.repository.finance.finance_repo import FinanceRepo
%load_ext autoreload
%autoreload 2

from backtest.domains.strategy import StockData
from backtest.domains.strategy import Strategy
from backtest.domains.strategy_result import StrategyResult,StrategyResultColumnType
from backtest.domains.backtest import Backtest
from backtest.repository.webrepo.bithumb_repo import BithumbRepo
from backtest.repository.webrepo.upbit_repo import UpbitRepo
from backtest.repository.webrepo.binance_repo import BinanceRepo
from backtest.use_cases.stockdata_from_repo import stockdata_from_repo
from backtest.request.stockdata_from_repo import build_stock_data_from_repo_request
from backtest.use_cases.strategy_execute import rsi_function, rsi_big_stock_function, greed_fear_index_function
from backtest.use_cases.backtest_execute import backtest_execute
from backtest.util.stockdata_util import get_greed_fear_index
import pandas as pd

request = build_stock_data_from_repo_request(
    filters={'order__eq': 'ALGO', 'from__eq': '2019-01-01'})
response = stockdata_from_repo(BithumbRepo(), request=request)

request2 = build_stock_data_from_repo_request(
    filters={'order__eq': 'BTC', 'from__eq': '2019-01-01'})
response2 = stockdata_from_repo(BithumbRepo(), request=request2)

request3 = build_stock_data_from_repo_request(
    filters={'order__eq': 'ETH', 'from__eq': '2019-01-01'})
response3 = stockdata_from_repo(BithumbRepo(), request=request3)

request5 = build_stock_data_from_repo_request(
    filters={'order__eq': 'ADA', 'from__eq': '2019-01-01'})
response5 = stockdata_from_repo(BithumbRepo(), request=request5)

request4 = build_stock_data_from_repo_request(
    filters={'order__eq': 'US500', 'from__eq': '2019-01-01'})
response4 = stockdata_from_repo(FinanceRepo(), request=request4)

stockdata1 = response.value
stockdata2 = response2.value
stockdata3 = response3.value
stockdata4 = response4.value
stockdata5 = response5.value
greed_fear_df = get_greed_fear_index()

strategy1 = Strategy(name='snp_big_stock', function=rsi_big_stock_function,
                    weight=3, options={'big_stock': stockdata4, 'rolling': 100})
strategy2 = Strategy(name='sma_big_stock', function=rsi_big_stock_function,
                    weight=2, options={'big_stock': stockdata2, 'rolling': 90})
strategy3 = Strategy(name='sma_self', function=rsi_function,
                    weight=1, options={'rolling': 7})
strategy4 = Strategy(name='Greed_Fear_Index',function=greed_fear_index_function,
                    weight=1, options={'greed_fear_index_data': greed_fear_df,'index_fear' : 20,'index_greed':60})
backtest = Backtest(strategy_list=[strategy1,strategy2,strategy3,strategy4], stockdata_list=[stockdata1,stockdata2,stockdata3,stockdata5])
backtest_result = backtest_execute(backtest).value
backtest_result.value
backtest_result.value['total_profit'].expanding().sum()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


date
2019-01-01    0.000000
2019-01-02    0.000000
2019-01-03    0.000000
2019-01-04    0.000000
2019-01-05    0.000000
                ...   
2023-03-16    3.770993
2023-03-17    3.770993
2023-03-18    3.770993
2023-03-19    3.770993
2023-03-20    3.770993
Freq: D, Name: total_profit, Length: 1540, dtype: float64

date
2019-01-01    0.000000
2019-01-02    0.000000
2019-01-03    0.000000
2019-01-04    0.000000
2019-01-05    0.000000
                ...   
2023-03-16    0.530775
2023-03-17    0.530775
2023-03-18    0.530775
2023-03-19    0.530775
2023-03-20    0.530775
Freq: D, Name: total_profit, Length: 1540, dtype: float64

In [125]:
from datetime import datetime

def genrate_empty_stockData(from_date,to_date=datetime.now().strftime('%Y-%m-%d'),symbol=''):
    date_series=pd.date_range(from_date,to_date)
    df=pd.DataFrame(columns=['open', 'high', 'low', 'close',
                                               'volume'],index=date_series).fillna(0)
    return StockData(symbol=symbol,data=df)
empty_stock=genrate_empty_stockData('2019-01-01')

d1=empty_stock.data
request = build_stock_data_from_repo_request(
    filters={'order__eq': 'ALGO', 'from__eq': '2019-01-01'})
response = stockdata_from_repo(BithumbRepo(), request=request)
stockdata1 = response.value
stockdata1.data=(d1+stockdata1.data).fillna(0)
stockdata1.data

,open,high,low,close,volume
date,,,,,
2020-07-30,360.0,447.0,360.0,391.0,1.237890e+08
2020-07-31,391.0,429.1,380.5,385.8,1.087587e+08
2020-08-01,385.2,394.0,373.0,389.0,1.073324e+08
2020-08-02,389.0,398.0,230.3,357.3,7.041321e+06
2020-08-03,356.5,384.1,351.6,370.8,3.398765e+06
...,...,...,...,...,...
2023-02-25,346.9,347.9,331.0,334.4,1.846893e+06
2023-02-26,334.7,338.3,326.3,334.9,1.488152e+06
2023-02-27,335.1,344.5,332.2,344.5,1.041184e+06


,open,high,low,close,volume
2019-01-01,0.0,0.0,0.0,0.0,0.000000e+00
2019-01-02,0.0,0.0,0.0,0.0,0.000000e+00
2019-01-03,0.0,0.0,0.0,0.0,0.000000e+00
2019-01-04,0.0,0.0,0.0,0.0,0.000000e+00
2019-01-05,0.0,0.0,0.0,0.0,0.000000e+00
...,...,...,...,...,...
2023-02-25,346.9,347.9,331.0,334.4,1.846893e+06
2023-02-26,334.7,338.3,326.3,334.9,1.488152e+06
2023-02-27,335.1,344.5,332.2,344.5,1.041184e+06
2023-02-28,345.0,346.4,327.8,329.2,1.237138e+06


In [62]:
pd.options.display.max_colwidth = 1000
backtest_result.value[['total_profit','total_stock_count','stock_count']].to_csv(
    'result.csv')


AttributeError: 'dict' object has no attribute 'data'

In [11]:
from backtest.repository.finance.finance_repo import FinanceRepo
request4 = build_stock_data_from_repo_request(
    filters={'order__eq': 'US500', 'from__eq': '2019-01-01'})
response4 = stockdata_from_repo(FinanceRepo(), request=request4)
stockdata4 = response4.value
stockdata4.data


NameError: name 'build_stock_data_from_repo_request' is not defined

In [18]:
stockdata2.data.join(df,how='inner')

,open,high,low,close,volume,sma,smashift,result,total,total_profit,stock_bucket,value,value_classification
date,,,,,,,,,,,,,
2019-01-01,4265000.0,4278000.0,4135000.0,4179000.0,2954.479096,0.000000e+00,0.000000e+00,"(StrategyResultColumnType.KEEP, 2)",StrategyResultColumnType.KEEP,0.0,[],24,Extreme Fear
2019-01-02,4176000.0,4328000.0,4137000.0,4302000.0,3802.249458,0.000000e+00,0.000000e+00,"(StrategyResultColumnType.KEEP, 2)",StrategyResultColumnType.KEEP,0.0,[],30,Fear
2019-01-03,4300000.0,4379000.0,4285000.0,4310000.0,15636.939649,0.000000e+00,0.000000e+00,"(StrategyResultColumnType.KEEP, 2)",StrategyResultColumnType.KEEP,0.0,[],33,Fear
2019-01-04,4305000.0,4322000.0,4238000.0,4262000.0,19589.941342,0.000000e+00,0.000000e+00,"(StrategyResultColumnType.KEEP, 2)",StrategyResultColumnType.KEEP,0.0,[],48,Neutral
2019-01-05,4261000.0,4356000.0,4250000.0,4299000.0,24952.263114,0.000000e+00,0.000000e+00,"(StrategyResultColumnType.KEEP, 2)",StrategyResultColumnType.KEEP,0.0,[],36,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-16,32985000.0,33420000.0,32107000.0,33099000.0,4076.663721,2.701918e+07,2.690252e+07,"(StrategyResultColumnType.BUY, 2)",StrategyResultColumnType.BUY,0.0,"[(BTC, 2023-02-05 00:00:00), (BTC, 2023-02-06 ...",52,Neutral
2023-03-17,33092000.0,35803000.0,32849000.0,34912000.0,5021.945872,2.716198e+07,2.701918e+07,"(StrategyResultColumnType.BUY, 2)",StrategyResultColumnType.BUY,0.0,"[(BTC, 2023-02-05 00:00:00), (BTC, 2023-02-06 ...",51,Neutral
2023-03-18,34912000.0,36666000.0,34853000.0,36348000.0,4226.135895,2.731966e+07,2.716198e+07,"(StrategyResultColumnType.BUY, 2)",StrategyResultColumnType.BUY,0.0,"[(BTC, 2023-02-05 00:00:00), (BTC, 2023-02-06 ...",64,Greed
